# Lesson 1: Simple ReAct Agent from Scratch
# based on https://til.simonwillison.net/llms/python-react-pattern

Lesson inspired in paper: https://arxiv.org/pdf/2210.03629



In [7]:
!pip install -q -r requirements.txt 

## Build an agent

In [21]:
import os 
import re 
import httpx
import openai
from openai import OpenAI

client = openai.OpenAI(api_key='sk-proj-5ne45azAu2oQ0FwD9vmAT3BlbkFJtX54vY2F160vafTu5Dbc')

In [31]:
chat_completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "Hello world"}]
)

In [32]:
chat_completion.choices[0].message.content


'Hello! How can I assist you today?'

In [34]:
# Building Agent
class Agent:
    def __init__(self, system=""):
        self.system =system 
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result 
    
    def execute(self):
        completion = client.chat.completions.create(
            model="gpt-4o-mini", 
            temperature=0.5,
            messages=self.messages)
        return completion.choices[0].message.content 


In [46]:
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in "Scottish Terrier":
        return ("Scottish Terrier average weight is 20 lbs")
    elif name in "Border Collie":
        return ("Border Collie average weight is 37 lbs")
    elif name in "Toy Poodle":
        return ("Toy Poodle average weight is 7 lbs")
    else:
        return ("An average dog weights 50 lbs")
    
known_actions = {
    "calculate": calculate, 
    "average_dog_weight": average_dog_weight
}

In [47]:
abot = Agent(prompt)

In [48]:
result = abot("How much does a toy poodle weight?")
print(result)

Thought: I need to find the average weight of a Toy Poodle using the average_dog_weight action. 
Action: average_dog_weight: Toy Poodle
PAUSE


In [49]:
result = average_dog_weight("Toy Poodle")
print(result)

Toy Poodle average weight is 7 lbs


In [50]:
next_prompt = "Observation: {}.format(result)"
abot(next_prompt)

'Answer: The average weight of a Toy Poodle is typically around 6 to 9 lbs.'

In [51]:
abot.messages

[{'role': 'system',
  'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\naverage_dog_weight:\ne.g. average_dog_weight: Collie\nreturns average weight of a dog when given the breed\n\nExample session:\n\nQuestion: How much does a Bulldog weigh?\nThought: I should look the dogs weight using average_dog_weight\nAction: average_dog_weight: Bulldog\nPAUSE\n\nYou will be called again with this:\n\nObservation: A Bulldog weights 51 lbs\n\nYou then output:\n\nAnswer: A bulldog weights 51 lbs'},
 {'role': 'user', 'content': 'How much does a 

In [52]:
abot =  Agent(prompt)

In [53]:
question = """I have 2 dogs, a border collie and scottish terrier. \ What is their combined weight?"""
abot(question)

"Thought: To find the combined weight of the two dogs, I need to determine the average weight of each breed first, then sum those weights together. I'll start by getting the average weight for the Border Collie and the Scottish Terrier. \n\nAction: average_dog_weight: Border Collie\nPAUSE"

In [54]:
next_prompt = "Observation: {}".format(average_dog_weight("Border Collie"))
print(next_prompt)
abot(next_prompt)

Observation: Border Collie average weight is 37 lbs


'Thought: Now that I have the average weight of the Border Collie, I need to find the average weight of the Scottish Terrier to calculate their combined weight.\n\nAction: average_dog_weight: Scottish Terrier\nPAUSE'

In [55]:
next_prompt = "Observation: {}".format(average_dog_weight("Scottish Terrier"))
print(next_prompt)
abot(next_prompt)

Observation: Scottish Terrier average weight is 20 lbs


'Thought: I now have the average weights for both dogs: the Border Collie weighs 37 lbs and the Scottish Terrier weighs 20 lbs. To find their combined weight, I will add these two weights together.\n\nAction: calculate: 37 + 20\nPAUSE'

In [56]:
next_prompt = "Observation: {}".format(calculate("37 + 20")) 
print(next_prompt)
abot(next_prompt)

Observation: 57


'Answer: The combined weight of a Border Collie and a Scottish Terrier is 57 lbs.'

## Bulding ReAct Agent

In [70]:
## Prompt: You can change this for a different pourpose
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

average_dog_weight:
e.g. average_dog_weight: Collie
returns average weight of a dog when given the breed

Example session:

Question: How much does a Bulldog weigh?
Thought: I should look the dogs weight using average_dog_weight
Action: average_dog_weight: Bulldog
PAUSE

You will be called again with this:

Observation: A Bulldog weights 51 lbs

You then output:

Answer: A bulldog weights 51 lbs
""".strip()

## Helper functions: Need to be aligned with the problem prompt
def calculate(what):
    return eval(what)

def average_dog_weight(name):
    if name in "Scottish Terrier":
        return ("Scottish Terrier average weight is 20 lbs")
    elif name in "Border Collie":
        return ("Border Collie average weight is 37 lbs")
    elif name in "Toy Poodle":
        return ("Toy Poodle average weight is 7 lbs")
    else:
        return ("An average dog weights 50 lbs")
    
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
            model="gpt-4o-mini", 
            temperature=0.5,
            messages=self.messages)
        return completion.choices[0].message.content 

# ReActAgent Class
class ReActAgent(Agent):
    def __init__(self, system=""):
        super().__init__(system)
        self.action_re = re.compile(r'^Action: (\w+): (.*)$')
        self.known_actions = {
            "calculate": calculate,
            "average_dog_weight": average_dog_weight
        }

    def process(self, prompt, question, max_turn=5):
        i = 0
        next_prompt = question
        while i < max_turn:
            i += 1
            result = self(next_prompt)
            print(result)
            actions = [self.action_re.match(a) for a in result.split('\n') if self.action_re.match(a)]
            if actions:
                action, action_input = actions[0].groups()
                if action not in self.known_actions:
                    raise Exception(f"Unknown action: {action}: {action_input}")
                print(f"-- running {action} {action_input}")
                observation = self.known_actions[action](action_input)
                print("Observation:", observation)
                next_prompt = f"Observation: {observation}"
            else:
                return

In [69]:
react_agent = ReActAgent()
question = """I have 2 dogs, a border collie and a scottish terrier. What is their combined weight"""

react_agent.process(prompt, f"Question: {question}")

The combined weight of your two dogs, a Border Collie and a Scottish Terrier, can vary based on their individual sizes and health. On average, a Border Collie typically weighs between 30 to 45 pounds (13 to 20 kg), while a Scottish Terrier usually weighs around 18 to 22 pounds (8 to 10 kg). 

If we take an average weight for each breed, we can estimate:

- Border Collie: approximately 37.5 pounds (17 kg)
- Scottish Terrier: approximately 20 pounds (9 kg)

Adding these together gives a combined weight of about 57.5 pounds (26 kg). 

For a more accurate estimate, you would need to weigh each dog individually.
